## Christoffel Symbols Calculator

**NOTE**: Use geometrized units ($c = G = 1$).

In [1]:
from IPython.display import display, clear_output
from itertools import product
import numpy as np
import sympy as sp
from time import sleep

### Obtain a valid number of coordinates.

In [3]:
while(True):
    n = input("\nEnter the number of coordinates: ")
    try:
        n = int(n)
        if n > 0:
            break
    except ValueError:
            pass
    print("\nNumber of coordinates must be a positive integer! Try again.\n")
    clear_output(wait=True)


Enter the number of coordinates: 4


### Enter the desired coordinates.

**NOTE**: To obtain coordinates like ($r$, $\theta$), type **r theta** when prompted. 

In [4]:
while(True):
    coordinates = [i for i in input("\nEnter the coordinates: ").split()]
    x = sp.symbols(coordinates)
    print("\nThe coordinates are:")
    display(sp.Eq(sp.Matrix(1,n,sp.symbols([f"x^{i}" for i in range(n)])), sp.Matrix(1,n,x)))
    choice = input("\nProceed with these coordinates? [y/n]: ").lower()
    if choice == 'y':
        break
    clear_output(wait=True)


Enter the coordinates: t r theta phi

The coordinates are:


Eq(Matrix([[x^0, x^1, x^2, x^3]]), Matrix([[t, r, theta, phi]]))


Proceed with these coordinates? [y/n]: y


### Generate the symbolic notations for the Christoffel symbols.

In [5]:
# Create array to store the symbolic notations of the christoffel symbols.
gamma_list = np.zeros(shape=n, dtype='object')

for i in range(n): 
    dummy_matrix = sp.Matrix.zeros(n,n)
    for (j,k) in product(range(n), repeat=2):
        dummy_matrix[j,k] = sp.Symbol(f"\Gamma^{x[i]}_{x[j]}_{x[k]}")
    gamma_list[i] = dummy_matrix

### Enter the components of the covariant metric tensor. 

**NOTE**: Input the terms in the usual Python format. For example, type **r\*\*2 * sin(theta)** for $r^2\sin(\theta)$.

In [7]:
# Create a matrix to store the covariant metric tensor components.
covariant_metric = sp.Matrix.zeros(n,n)

# Matrix g will be used to store the symbolic notations of the components.
g = covariant_metric.copy()

while(True):
    for i in range(n):
        for j in range(i,n):
            print("\nEnter the components of the symmetric, covariant metric tensor:")
            metric_component = sp.Symbol(f"\tg_{sp.latex(x[i])}_{sp.latex(x[j])}")
            display(metric_component)
            g[i,j] = metric_component
            sleep(0.5) # counter asynchronocity between display and input function calls.
            covariant_metric[i,j] = input("")
            if i!=j:
                g[j,i] = g[i,j]
                covariant_metric[j,i] = covariant_metric[i,j]
            clear_output(wait=True)
                
# Compute the contravariant metric tensor from the covariant metric tensor.
    try:
        contravariant_metric = covariant_metric.inv()
        break
    except sp.matrices.common.NonInvertibleMatrixError:
        print("\nThe matrix containing the covariant metric components is not invertible! Try again.")

print("\nThe covariant metric components are:")
display(sp.Eq(g,covariant_metric))


The covariant metric components are:


Eq(Matrix([
[     	g_t_t,      	g_t_r,      	g_t_\theta,      	g_t_\phi],
[     	g_t_r,      	g_r_r,      	g_r_\theta,      	g_r_\phi],
[	g_t_\theta, 	g_r_\theta, 	g_\theta_\theta, 	g_\theta_\phi],
[  	g_t_\phi,   	g_r_\phi,   	g_\theta_\phi,   	g_\phi_\phi]]), Matrix([
[1/(1 - 2*Phi(r)/c**2),                 0, 0, 0],
[                    0, 1 - 2*Phi(r)/c**2, 0, 0],
[                    0,                 0, 1, 0],
[                    0,                 0, 0, 1]]))

### Compute the Christoffel symbols.

In [8]:
# Create array to store the computed christoffel symbols.
christoffel_symbols = np.zeros(shape=n, dtype='object')

for i in range(n):
    dummy_matrix = sp.Matrix.zeros(n,n)
    for (j,k,l) in product(range(n), repeat=3):
        dummy_matrix[j,k] += (
            sp.Rational(1/2)*contravariant_metric[i,l] * (sp.diff(covariant_metric[l,j],x[k]) 
            +sp.diff(covariant_metric[l,k],x[j]) - sp.diff(covariant_metric[j,k],x[l]))
        )
    christoffel_symbols[i] = sp.simplify(dummy_matrix)

The Christoffel symbols are:

In [9]:
for i in range(n):
    display(sp.Eq(gamma_list[i], christoffel_symbols[i]))

Eq(Matrix([
[    \Gamma^t_t_t,     \Gamma^t_t_r,     \Gamma^t_t_theta,     \Gamma^t_t_phi],
[    \Gamma^t_r_t,     \Gamma^t_r_r,     \Gamma^t_r_theta,     \Gamma^t_r_phi],
[\Gamma^t_theta_t, \Gamma^t_theta_r, \Gamma^t_theta_theta, \Gamma^t_theta_phi],
[  \Gamma^t_phi_t,   \Gamma^t_phi_r,   \Gamma^t_phi_theta,   \Gamma^t_phi_phi]]), Matrix([
[                                      0, Derivative(Phi(r), r)/(c**2 - 2*Phi(r)), 0, 0],
[Derivative(Phi(r), r)/(c**2 - 2*Phi(r)),                                       0, 0, 0],
[                                      0,                                       0, 0, 0],
[                                      0,                                       0, 0, 0]]))

Eq(Matrix([
[    \Gamma^r_t_t,     \Gamma^r_t_r,     \Gamma^r_t_theta,     \Gamma^r_t_phi],
[    \Gamma^r_r_t,     \Gamma^r_r_r,     \Gamma^r_r_theta,     \Gamma^r_r_phi],
[\Gamma^r_theta_t, \Gamma^r_theta_r, \Gamma^r_theta_theta, \Gamma^r_theta_phi],
[  \Gamma^r_phi_t,   \Gamma^r_phi_r,   \Gamma^r_phi_theta,   \Gamma^r_phi_phi]]), Matrix([
[-c**4*Derivative(Phi(r), r)/(c**2 - 2*Phi(r))**3,                                        0, 0, 0],
[                                               0, -Derivative(Phi(r), r)/(c**2 - 2*Phi(r)), 0, 0],
[                                               0,                                        0, 0, 0],
[                                               0,                                        0, 0, 0]]))

Eq(Matrix([
[    \Gamma^theta_t_t,     \Gamma^theta_t_r,     \Gamma^theta_t_theta,     \Gamma^theta_t_phi],
[    \Gamma^theta_r_t,     \Gamma^theta_r_r,     \Gamma^theta_r_theta,     \Gamma^theta_r_phi],
[\Gamma^theta_theta_t, \Gamma^theta_theta_r, \Gamma^theta_theta_theta, \Gamma^theta_theta_phi],
[  \Gamma^theta_phi_t,   \Gamma^theta_phi_r,   \Gamma^theta_phi_theta,   \Gamma^theta_phi_phi]]), Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]]))

Eq(Matrix([
[    \Gamma^phi_t_t,     \Gamma^phi_t_r,     \Gamma^phi_t_theta,     \Gamma^phi_t_phi],
[    \Gamma^phi_r_t,     \Gamma^phi_r_r,     \Gamma^phi_r_theta,     \Gamma^phi_r_phi],
[\Gamma^phi_theta_t, \Gamma^phi_theta_r, \Gamma^phi_theta_theta, \Gamma^phi_theta_phi],
[  \Gamma^phi_phi_t,   \Gamma^phi_phi_r,   \Gamma^phi_phi_theta,   \Gamma^phi_phi_phi]]), Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]]))

In [13]:
from sympy import symbols, Function

t, r, theta, phi, epsilon_0, lambda_ = symbols('t r theta phi epsilon_0 lambda_ ')  # Redefining symbols
phi_function = Function('phi')(r)  # Redefining phi as a function of r


# Define the gravitational potential as a function of r only
phi = sp.Function('phi')(r)

# Define components of the wave vector assuming radial symmetry for simplicity
kappa_r = sp.symbols('kappa_r')

# Redefine the derivative of epsilon^0
d_epsilon_0_dlambda_simple = sp.Eq(sp.diff(epsilon_0, lambda_), -2 * kappa_r * sp.diff(phi, r))

# Display the equation for d(epsilon^0)/d(lambda)
d_epsilon_0_dlambda_simple


Eq(0, -2*kappa_r*Derivative(phi(r), r))